# (Title to be decided)

*This is the file for the code of our group assignment. Under development.*

In [1]:
import json
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

## 1. Read Data from External File

TODO: Still Under Dev

Read data in external files. For convenience, the original `.pcapng` file has been converted into `.json` and `.csv` with Wireshark.

In [2]:
# Read the JSON including all details of packets
#   errors="surrogateescape" to pass the encoding errors raised by contents in HTTP packets
with open("./data/traffic.json", "r", encoding="utf-8", errors="surrogateescape") as f:
    dataset_json=json.load(f)
    print(f"{len(dataset_json)} packets read")

# print one for checking
dataset_json[0]

10625 packets read


{'_index': 'packets-XXXX-XX-XX',
 '_type': 'doc',
 '_score': None,
 '_source': {'layers': {'pkt_comment': {'frame.comment': '13682128230572000042,china',
    'frame.comment_tree': {'_ws.expert': {'frame.comment.expert': '',
      '_ws.expert.message': '13682128230572000042,china',
      '_ws.expert.severity': '1048576',
      '_ws.expert.group': '184549376'}}},
   'frame': {'frame.interface_id': '0',
    'frame.interface_id_tree': {'frame.interface_name': 'unknown'},
    'frame.encap_type': '7',
    'frame.time': 'Not representable',
    'frame.offset_shift': '0.000000000',
    'frame.time_epoch': '-1460980371.753735000',
    'frame.time_delta': '0.000000000',
    'frame.time_delta_displayed': '0.000000000',
    'frame.time_relative': '0.000000000',
    'frame.number': '1',
    'frame.len': '58',
    'frame.cap_len': '58',
    'frame.marked': '0',
    'frame.ignored': '0',
    'frame.protocols': 'raw:ip:udp:dns',
    'frame.coloring_rule.name': 'UDP',
    'frame.coloring_rule.string': 

In [3]:
# Read the CSV
# TODO: Info in CSV should be covered by the one in JSON.
#       Remove it in the future if not used
dataset_csv=pd.read_csv("./data/traffic.csv")

dataset_csv

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000e+00,10.11.1.3,dns.google,DNS,58,Standard query 0xe8e9 A lcs.naver.jp
1,2,1.234100e-02,dns.google,10.11.1.3,DNS,74,Standard query response 0xe8e9 A lcs.naver.jp ...
2,3,3.187300e-02,10.11.1.3,dns.google,DNS,64,Standard query 0x1556 A cgi.connect.qq.com
3,4,3.673000e-02,10.11.1.3,dns.google,DNS,59,Standard query 0x0d1f A fusion.qq.com
4,5,1.151130e-01,10.11.1.3,dns.google,DNS,68,Standard query 0x639b A nelo2-col.linecorp.com
...,...,...,...,...,...,...,...
10620,10621,7.349290e+08,10.11.2.4,e7178.g.akamaiedge.net,TCP,52,59035 > 80 [ACK] Seq=652 Ack=5079 Win=93504 ...
10621,10622,7.349290e+08,e7178.g.akamaiedge.net,10.11.2.4,HTTP,211,HTTP/1.1 200 OK
10622,10623,7.349290e+08,10.11.2.4,dns.google,DNS,59,Standard query 0x321a AAAA api.zedge.net
10623,10624,7.349290e+08,dns.google,10.11.2.4,DNS,129,Standard query response 0x321a AAAA api.zedge....


## 2. Data Preprocessing

TODO: Still Under Dev

### 2.1 Extract dataset labels

In [4]:
# Extract labels(y) from JSON
dataset_y = np.array(list(map(
    lambda packet_json: (packet_json["_source"]["layers"]["pkt_comment"]["frame.comment"]).split(",")[1], 
    dataset_json
)))

print(f"Labels include {np.unique(dataset_y)}")
print(dataset_y.shape)

Labels include ['china' 'india' 'us']
(10625,)


### 2.2 Extract features

In [5]:
# Extract source IP, source IP host, destination IP and destination IP host from dataset
dataset_X_ip_src = np.array(list(map(
    lambda packet_json: packet_json["_source"]["layers"]["ip"]["ip.src"], 
    dataset_json
)))
dataset_X_ip_src_host = np.array(list(map(
    lambda packet_json: packet_json["_source"]["layers"]["ip"]["ip.src_host"], 
    dataset_json
)))
dataset_X_ip_dst = np.array(list(map(
    lambda packet_json: packet_json["_source"]["layers"]["ip"]["ip.dst"], 
    dataset_json
)))
dataset_X_ip_dst_host = np.array(list(map(
    lambda packet_json: packet_json["_source"]["layers"]["ip"]["ip.dst_host"], 
    dataset_json
)))

# print first ones for checking
print(dataset_X_ip_src[0])
print(dataset_X_ip_src_host[0])
print(dataset_X_ip_dst[0])
print(dataset_X_ip_dst_host[0])


10.11.1.3
10.11.1.3
8.8.8.8
dns.google


In [6]:
# Encode nominal features to numeric for testing with kNN
#
# TODO: Temp added for kNN testing - not an apporpriate way for handling strings.
#       The transformation here does not help finding the similarity between them.
def encode_label(original):
    le = preprocessing.LabelEncoder()
    le.fit(original)
    return le.transform(original)

dataset_X_ip_src=encode_label(dataset_X_ip_src)
dataset_X_ip_src_host=encode_label(dataset_X_ip_src_host)
dataset_X_ip_dst=encode_label(dataset_X_ip_dst)
dataset_X_ip_dst_host=encode_label(dataset_X_ip_dst_host)

In [7]:
# Collect extracted features to one place
dataset_X = pd.DataFrame({
    "ip_src": dataset_X_ip_src,
    "ip_src_host": dataset_X_ip_src_host,
    "ip_dst": dataset_X_ip_dst,
    "ip_dst_host": dataset_X_ip_dst_host,
})

print(dataset_X.shape)
dataset_X.head()

(10625, 4)


,ip_src,ip_src_host,ip_dst,ip_dst_host
0,1,3,740,243
1,631,186,1,3
2,1,3,740,243
3,1,3,740,243
4,1,3,740,243


In [8]:
# Split the dataset into training set and testing set
#
# TODO: validation set not created here - might be needed? 
#       just add it later if it is necessary in the future
train_X, test_X, train_y, test_y = train_test_split(dataset_X, dataset_y, random_state=123)

print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(7968, 4)
(7968,)
(2657, 4)
(2657,)


In [9]:
# kNN classification test
#
# TODO: A brief test for evaluating performance during writing preprocessin code, 
#       gonna be removed later
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_X, train_y)
print(f"Accuracy on test set: {knn.score(test_X, test_y):.4f}")


Accuracy on test set: 0.8220


## 3. Model Training

### 3.1 Logistic Regression

TODO

### 3.2 Support Vector Machine (SVM)

TODO

### 3.3 (?) neural network

Maybe?

## 4. Model Testing and Comparison

TODO